#### First let's import a few things

In [1]:
import os
import ceci
from ceci.pipeline import Pipeline

#### For now we need to be in the top-level ceci directory to read the input yaml file

This is b/c it refers to the configuration file, which we will also need to read.

We should fix this.

In [2]:
os.chdir(os.path.join(os.path.dirname(ceci.__file__), '..'))
pipeline = Pipeline.read('tests/test.yml')

#### Now we have some pipeline introspection

1. Printing a list of stages
2. Accessing a stage as an attribute of the pipeline
3. Printing the inputs and output connections associated to a stage
4. Printing the configuration associated to a stage

In [3]:
pipeline.print_stages()

PZEstimationPipe    : <ceci_example.example_stages.PZEstimationPipe object at 0x7fdb24578280>
shearMeasurementPipe: <ceci_example.example_stages.shearMeasurementPipe object at 0x7fdb247011f0>
WLGCSelector        : <ceci_example.example_stages.WLGCSelector object at 0x7fdb24701250>
SysMapMaker         : <ceci_example.example_stages.SysMapMaker object at 0x7fdb2470dcd0>
SourceSummarizer    : <ceci_example.example_stages.SourceSummarizer object at 0x7fdb2470d460>
WLGCCov             : <ceci_example.example_stages.WLGCCov object at 0x7fdb2470ddc0>
WLGCRandoms         : <ceci_example.example_stages.WLGCRandoms object at 0x7fdb2470d130>
WLGCTwoPoint        : <ceci_example.example_stages.WLGCTwoPoint object at 0x7fdb2470d970>
WLGCSummaryStatistic: <ceci_example.example_stages.WLGCSummaryStatistic object at 0x7fdb2470de20>


In [4]:
print(pipeline.SysMapMaker)

In [5]:
pipeline.SysMapMaker.print_io()

Inputs--------
DM                   : <class 'ceci_example.types.TextFile'> : ./tests/inputs/dm.txt
Outputs--------
diagnostic_maps      : <class 'ceci_example.types.TextFile'> : diagnostic_maps.txt


In [7]:
print(repr(pipeline.WLGCSelector.config))

StageConfig{zbin_edges:[0.2, 0.3, 0.5],ra_range:[-5, 5],metacalibration:True,}


#### Ok, eventually this example should include most stuff, but for now let's just the pipeline

In [8]:
status = pipeline.run()


Executing PZEstimationPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example PZEstimationPipe   --DM=./tests/inputs/dm.txt   --fiducial_cosmology=./tests/inputs/fiducial_cosmology.txt   --config=./tests/config.yml   --photoz_pdfs=./tests/outputs/photoz_pdfs.txt 
Output writing to ./tests/logs/PZEstimationPipe.out

Job PZEstimationPipe has completed successfully!

Executing shearMeasurementPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example shearMeasurementPipe   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --shear_catalog=./tests/outputs/shear_catalog.txt 
Output writing to ./tests/logs/shearMeasurementPipe.out

Job shearMeasurementPipe has completed successfully!

Executing WLGCSelector
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example WLGCSelector   --shear_catalog=./tests/outputs/shear_catalog.txt   --photoz_pdfs=./tests/outputs/photoz_pdfs.txt   --config=./tests/config.yml   --tomography_catalog=./tests/outputs/tomography_catalog.txt 
Output writ

#### Now let's do the same thing by building the pipeline with the python interface

In [9]:
from ceci_example.example_stages import *

In [10]:
pipe2 = Pipeline.interactive()
overall_inputs = {'DM':'./tests/inputs/dm.txt',
                  'fiducial_cosmology':'./tests/inputs/fiducial_cosmology.txt'}
inputs = overall_inputs.copy()
inputs['metacalibration'] = True
inputs['config'] = None

inputs.update(pipe2.add_stage(PZEstimationPipe.build(**inputs)))
inputs.update(pipe2.add_stage(shearMeasurementPipe.build(**inputs, apply_flag=False)))
inputs.update(pipe2.add_stage(WLGCSelector.build(**inputs, zbin_edges=[0.2, 0.3, 0.5], ra_range=[-5, 5])))
inputs.update(pipe2.add_stage(SysMapMaker.build(**inputs)))
inputs.update(pipe2.add_stage(SourceSummarizer.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCCov.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCRandoms.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCTwoPoint.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCSummaryStatistic.build(**inputs)))


In [ ]:
pipe2.build_stage(PZEstimationPipe, zbin_edges=[], ra_range)

In [11]:
pipe2.load_configs(overall_inputs, pipeline.run_config, pipeline.stages_config)

(({'SysMapMaker': <Job SysMapMaker>,
   'WLGCRandoms': <Job WLGCRandoms>,
   'shearMeasurementPipe': <Job shearMeasurementPipe>,
   'PZEstimationPipe': <Job PZEstimationPipe>,
   'WLGCSelector': <Job WLGCSelector>,
   'WLGCTwoPoint': <Job WLGCTwoPoint>,
   'SourceSummarizer': <Job SourceSummarizer>,
   'WLGCCov': <Job WLGCCov>,
   'WLGCSummaryStatistic': <Job WLGCSummaryStatistic>},
   <ceci_example.example_stages.WLGCSummaryStatistic at 0x7fdb24740b50>]),
 {'output_dir': './tests/outputs', 'log_dir': './tests/logs', 'resume': False})

In [12]:
pipe2.WLGCSelector.config['zbin_edges'] = False

TypeError: Value of type <class 'bool'>, when <class 'list'> was expected.

In [13]:
pipe2.run()


Executing SysMapMaker
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example SysMapMaker   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --diagnostic_maps=./tests/outputs/diagnostic_maps.txt 
Output writing to ./tests/logs/SysMapMaker.out

Job SysMapMaker has completed successfully!

Executing WLGCRandoms
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example WLGCRandoms   --diagnostic_maps=./tests/outputs/diagnostic_maps.txt   --config=./tests/config.yml   --random_catalog=./tests/outputs/random_catalog.txt 
Output writing to ./tests/logs/WLGCRandoms.out

Job WLGCRandoms has completed successfully!

Executing shearMeasurementPipe
Command is:
OMP_NUM_THREADS=1   python3 -m ceci_example shearMeasurementPipe   --DM=./tests/inputs/dm.txt   --config=./tests/config.yml   --shear_catalog=./tests/outputs/shear_catalog.txt 
Output writing to ./tests/logs/shearMeasurementPipe.out

Job shearMeasurementPipe has completed successfully!

Executing PZEstimationPipe
Command is:
OMP_NU

0